In [2]:
import numpy as np

In [19]:
from ShallowMindBot import *
from TowerDefense import *
from TowerDefenseApi import *

In [20]:
towerDefense = TowerDefense('state.json')

In [22]:
api = TowerDefenseApi(towerDefense.getGameState())

In [23]:
bot = ShallowMindBot()

In [24]:
bot.doTurn(api)

Selected action type 2 from options [0, 1, 2, 6] with probabilities [0.24234084 0.24830685 0.25839406 0.25095826]


'4,0,2'

In [25]:
possible_actions = [0,1,2,6]
p, h = bot.policy_forward(bot.computeStateFeatures(api), possible_actions)

In [26]:
p

array([0.24234084, 0.24830685, 0.25839406, 0.25095826])

In [28]:
selected_action = 2

In [27]:
h

array([1.8721813 , 0.        , 0.        , 1.53647372, 0.        ,
       3.83066152, 0.        , 0.        , 1.03934245, 0.03170608,
       0.        , 0.        , 7.01988837, 0.        , 0.52261405])

In [4]:
batch_size = 10 # used to perform a RMS prop param update every batch_size steps
learning_rate = 1e-3 # learning rate used in RMS prop
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2

In [5]:
NUM_POSSIBLE_ACTIONS = 7

In [18]:
def softmax_grad(softmax):
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

In [82]:
selected_action = 0
dsoftmax = softmax_grad(p)[selected_action,:]
dsoftmax

array([ 0.18361176, -0.06017489, -0.06261943, -0.06081743])

In [30]:
dlog = dsoftmax / p[selected_action]
dlog

array([-0.24234084, -0.24830685,  0.74160594, -0.25095826])

In [31]:
x = np.array([float(val) for val in bot.computeStateFeatures(api).values()])

In [33]:
x

array([ 5.000e+01,  3.500e+02,  1.000e+02,  1.500e+01,  8.500e+01,
        5.200e+01,  1.300e+01,  3.900e+01,  2.081e+03,  0.000e+00,
        0.000e+00,  5.000e+00,  6.000e+00,  4.000e+00,  1.000e+00,
        1.000e+00, -2.000e+01, -2.000e+01,  0.000e+00,  0.000e+00,
        1.000e+00,  0.000e+00,  8.000e+00,  0.000e+00,  1.400e+01,
        0.000e+00,  6.000e+00,  3.000e+00])

In [35]:
# Fill in dlog with zeros at the actions that were not possible
dlog_filled = np.zeros(NUM_ACTION_TYPES)
dlog_filled[possible_actions] = dlog
dlog_filled

array([-0.24234084, -0.24830685,  0.74160594,  0.        ,  0.        ,
        0.        , -0.25095826])

In [52]:
h.shape

(15,)

In [53]:
dlog_filled.shape

(7,)

In [84]:
# Take outer product to get gradW2
grad_W2 = dlog_filled[None,:].T.dot(h[None,:])
# Confirm that the grad is 0 for the forbidden actions
grad_W2

array([[-0.45370598, -0.        , -0.        , -0.37235033, -0.        ,
        -0.92832572, -0.        , -0.        , -0.25187512, -0.00768368,
        -0.        , -0.        , -1.70120562, -0.        , -0.12665073],
       [-0.46487544, -0.        , -0.        , -0.38151694, -0.        ,
        -0.95117948, -0.        , -0.        , -0.25807585, -0.00787284,
        -0.        , -0.        , -1.74308634, -0.        , -0.12976865],
       [ 1.38842078,  0.        ,  0.        ,  1.13945804,  0.        ,
         2.84084134,  0.        ,  0.        ,  0.77078254,  0.02351342,
         0.        ,  0.        ,  5.20599092,  0.        ,  0.38757368],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , 

In [59]:
bot.model['W2'].shape

(7, 15)

In [85]:
grad_W2.shape

(7, 15)

In [119]:
def gradW2(p, possible_actions, selected_action, h):
    dsoftmax = softmax_grad(p)[selected_action,:]
    dlog = dsoftmax / p[selected_action]
    # Fill in dlog with zeros at the actions that were not possible
    dlog_filled = np.zeros(NUM_ACTION_TYPES)
    dlog_filled[possible_actions] = dlog
    dW2 = dlog_filled[None,:].T.dot(h[None,:])
    return dW2, dlog_filled

In [120]:
gradW2, dlog_filled = gradW2(p, possible_actions, selected_action, h)

In [118]:
def gradW1(dlog_filled, W2, h, x):
    dh = W2.T.dot(dlog_filled)
    dh[h<=0] = 0
    dW1 = np.outer(dh,x)
    return dW1

In [108]:
gradw1 = gradW1(dlog_filled, bot.model['W2'], h, x)

In [ ]:
def policy_backward(self, eph, epx, eplogp, eppossactions):
    """ backward pass. (eph is array of intermediate hidden states) """
    """ Manual implementation of a backward prop"""
    """ It takes an array of the hidden states that corresponds to all the images that were
    fed to the NN (for the entire episode, so a bunch of games) and their corresponding logp"""
    eplogp_filled = np.zeros(NUM_POSSIBLE_ACTIONS)
    eplogp_filled[epossactions] = eplogp
    dW2 = np.dot(eph.T, eplogp).ravel()
    dh = np.outer(eplogp, self.model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}